# Preprocessing
Because we have shown how to clean the data in a [different repository](https://github.com/frkncbngl/pandas-data-cleaning) we will explain only the differences and show you how you can create the main model that was trained, instead of the smaller version that was created due to GitHub's and Streamlit Cloud's file size limitations.

Now since we have more than 1 file, we need to find them and store them in list, after that we will do the something similar to what we did in the other repository. We will append our dataframe for every file and every sheet.

In [1]:
import pandas as pd
import os

def get_xlsx_files(directory):
    xlsx_files = []
    for file in os.listdir(directory):
        if file.endswith(".xlsx"):
            xlsx_files.append(file)
    return xlsx_files

directory = r"/path/folder"
xlsx_files = get_xlsx_files(directory)
print(xlsx_files)

['AMSSFO.xlsx', 'AMSCPT.xlsx', 'LHRDXB.xlsx', 'LHRNYC.xlsx', 'LHRBKK.xlsx', 'AMSKUL.xlsx', 'LHRSIN.xlsx', 'AMSSIN.xlsx', 'AMSTHR.xlsx', 'LHRIKA.xlsx', 'AMSBKK.xlsx', 'LHRKUL.xlsx', 'AMSNYC.xlsx', 'LHRCPT.xlsx', 'AMSDXB.xlsx', 'LHRSFO.xlsx']


In [2]:
list_of_dfs = []
for file in xlsx_files:
    file = pd.ExcelFile(file)
    for sheet in file.sheet_names:
        df = file.parse(sheet)
        list_of_dfs.append(df)
data = pd.concat(list_of_dfs,ignore_index= True)

In [3]:
data.drop("Advisory filter_alt",axis=1,inplace=True)

In [4]:
new_column_names = ["price","carrier","depart_time","arrive_time","duration","from_to","stops","start_date","end_date"]
data.columns = new_column_names

Something changes here, earlier we had only Euros as currency, but because of the data scraped now has Pounds, we need to convert them aswell. And we also need to use the current exchange rate while converting. Which is approx 1.17€ for 1 £.

In [5]:
def price_column_cleaner(row):
    if "£" in row:
        row = row.replace("£","")
        row = row.replace(",","")
        row = int(row)*1.17
        return int(row)
    elif "€" in row:
        row = row.replace("€","")
        row = row.replace(",","")
        return int(row)
    else:
        return int(row)

In [6]:
data["price"] = data["price"].apply(price_column_cleaner)

In [7]:
data["price"] = data["price"].astype(int)

In [8]:
data["stops_list"] = data["stops"].str.split()

In [9]:
def count_stops(row):
    #This function will be used in apply method. 
    # We use if/else conditions to check if the type of that "cell" is a list or not. 
    # If it is a list it means split method successfully seperated the entire column on a space, if not it still is a Nan. 
    # So we can check if the "cell" contains a list, and if so we can count the items within that list and return it as a stops count.
    
    if type(row["stops_list"]) == list:
        total_stops=len(row["stops_list"])
    else:
        total_stops = 0
    return total_stops

In [10]:
data["total_stops"] = data.apply(count_stops,axis=1)

In [11]:
data.drop(["arrive_time","stops","stops_list"],axis=1,inplace=True)

In [12]:
#This will return a dataframe with no column names.
depart = data["depart_time"].str.extract(r"(\d+:\d+ [APM]+)(\d+:\d+ [APM]+)")
depart_column_names = ["dep_time_outbound","dep_time_inbound"]
depart.columns = depart_column_names
data.drop("depart_time",axis=1,inplace=True)
data = pd.concat([data,depart],axis=1)

In [13]:
durations = data["duration"].str.extract(r"(\d+[h] \d+[m])( \d+[h] \d+[m])")
durations_column_names = ["duration_outbound","duration_inbound"]
durations.columns = durations_column_names
data.drop("duration",axis=1,inplace=True)
data = pd.concat([data,durations],axis=1)

In [14]:
from_to =data["from_to"].str.extract(r'([A-Z][A-Z]+) to ([A-Z][A-Z]+)')
from_to_column_names =  ["from","to"]
from_to.columns = from_to_column_names
data = pd.concat([data,from_to],axis=1)
data.drop("from_to",axis=1,inplace=True)

In [15]:
data["carrier"] = data["carrier"].apply(lambda x:x.strip().split(",")[0])

# Feature Engineering

In [16]:
data["day_of_week"] = pd.to_datetime(data["start_date"]).dt.day_of_week

In [17]:
def to_minutes(time_string):
  time_delta = pd.to_timedelta(time_string)
  minutes = time_delta.total_seconds() / 60
  return minutes

In [18]:
data["outbound_duration_minutes"] = data["duration_outbound"].apply(to_minutes)
data["inbound_duration_minutes"] = data["duration_inbound"].apply(to_minutes)

In [19]:
def to_time_of_day(time_string):
  hour = int(time_string.split(":")[0])  #Extracting hour from time string
  am_pm = time_string.split("AM")[-1]  #Extract AM/PM indicator

  if am_pm == "PM" and hour != 12:  #Adjusting for hours that represent PM.
    hour += 12

  #Classifying hours to time of days.
  if 5 <= hour < 12:
    return "Morning"
  elif 12 <= hour < 17:
    return "Afternoon"
  elif 17 <= hour < 20:
    return "Evening"
  elif 20 <= hour < 24:
    return "Night"
  else:
    return "Early Morning"

In [20]:
data["time_of_day_outbound"] = data["dep_time_outbound"].apply(to_time_of_day)
data["time_of_day_inbound"] = data["dep_time_inbound"].apply(to_time_of_day)

In [21]:
data["day_of_year"] = pd.to_datetime(data["start_date"]).dt.day_of_year

Important note! We convert differnt airports in the same city to city code. So;
- LGA, EWR, JFK to NYC
- XNB to DXB
- DMK to BKK

In [22]:
data["to"] = data["to"].replace("LGA","NYC")
data["to"] = data["to"].replace("EWR","NYC")
data["to"] = data["to"].replace("JFK","NYC")
data["to"] = data["to"].replace("XNB","DXB")
data["to"] = data["to"].replace("DMK","BKK")

More features

In [23]:
data["month_number"] = pd.to_datetime(data["start_date"]).dt.month
data["day_of_month"] = pd.to_datetime(data["start_date"]).dt.day
data["month_number_end"] = pd.to_datetime(data["end_date"]).dt.month
data["day_of_month_end"] = pd.to_datetime(data["end_date"]).dt.day

In [24]:
data.to_csv("data.csv",index=False)